In [2]:
import os
import torch
from torch import nn
from d2l import torch as d2l

### 加载预训练词向量

**以下列出维度为50、100和300的预训练GloVe嵌⼊，可从GloVe⽹站196下载。预训练的fastText嵌⼊有多种语
⾔。这⾥我们使⽤可以从fastText⽹站197下载300维度的英⽂版本（“wiki.en”）。**

In [3]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')
#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')
#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')
#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

**为了加载这些预训练的GloVe和fastText嵌⼊，我们定义了以下TokenEmbedding类。**

In [22]:
class TokenEmbedding:
    """Glove嵌入"""

    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_name)
        self.unkown_idx = 0
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe⽹站：https://nlp.stanford.edu/projects/glove/
        # fastText⽹站：https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的⾸⾏
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unkown_idx) for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [23]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [24]:
len(glove_6b50d)

400001

In [25]:
glove_6b50d.token_to_idx['handsome']

11634

In [26]:
glove_6b50d.idx_to_token[555]

'moved'

In [27]:
glove_6b50d.idx_to_vec[555]

tensor([-0.2653,  0.1592,  0.3131, -0.3739, -0.4297, -0.6655, -1.4784,  0.1557,
        -0.6496, -0.9377, -0.0174, -0.3385, -0.6669, -0.3230, -0.0048,  0.4699,
        -0.4723, -0.3673, -0.8564,  0.4733,  0.4578,  0.2049,  0.2690,  0.5137,
        -0.2641, -1.3882,  0.3729, -0.1138,  0.2555, -0.1724,  2.6382, -0.0786,
         0.0254, -0.6266,  0.0452, -0.2375, -0.2195,  0.3394,  0.3188,  0.0234,
        -0.0355, -0.1758,  0.3261, -0.6051, -0.2626,  0.5529,  0.0618, -1.3162,
        -0.2552, -0.1327])

### 应⽤预训练词向量

**使⽤加载的GloVe向量，我们将通过下⾯的词相似性和类⽐任务中来展⽰词向量的语义。**

#### 词相似度

**为了根据词向量之间的余弦相似性为输⼊词查找语义相似的词，我们实现了以下knn（k近
邻）函数。**

In [28]:
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1, )) / (
            torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
            torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

**然后，我们使⽤TokenEmbedding的实例embed中预训练好的词向量来搜索相似的词。**

In [29]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):
        print(f'{embed.idx_to_token[int(i)]}: cosine相似度={float(c):.3f}')

**glove_6b50d中预训练词向量的词表包含400000个词和⼀个特殊的未知词元。排除输⼊词和未知词元后，我
们在词表中找到与“chip”⼀词语义最相似的三个词。**

In [30]:
get_similar_tokens('chip', 3, glove_6b50d)

chips: cosine相似度=0.856
intel: cosine相似度=0.749
electronics: cosine相似度=0.749


**下⾯输出与“baby”和“beautiful”相似的词。**

In [31]:
get_similar_tokens('baby', 3, glove_6b50d)

babies: cosine相似度=0.839
boy: cosine相似度=0.800
girl: cosine相似度=0.792


In [32]:
get_similar_tokens('beautiful', 3, glove_6b50d)

lovely: cosine相似度=0.921
gorgeous: cosine相似度=0.893
wonderful: cosine相似度=0.830


### 词类⽐

**除了找到相似的词，我们还可以将词向量应⽤到词类⽐任务中。例如，“man”:“woman”::“son”:“daughter”是⼀个词的类⽐。“man”是对“woman”的类⽐，“son”是对“daughter”的类⽐。具体来说，词类⽐任务可以定义为：对于单词类⽐a : b :: c : d，给出前三个词a、b和c，找到d。⽤vec(w)表⽰词w的向量，为了完成这个类⽐，我们将找到⼀个词，其向量与vec(c) + vec(b) − vec(a)的结果最相似。**

In [40]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # 删除未知词

**让我们使⽤加载的词向量来验证“male-female”类⽐**

In [41]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'

**下⾯完成⼀个“⾸都-国家”的类⽐：“beijing”: “china”:: “tokyo”: “japan”。这说明了预训练词向量中的语义。**

In [46]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'

In [47]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [54]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'